<a href="https://colab.research.google.com/github/yekcel/AI-Agent-MRI-Image-Defacing/blob/main/Multi_agnet_deface_MRI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q ultralytics opencv-python google-generativeai pillow numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.5 MB/s eta 0:00:00


In [2]:
import os
from pathlib import Path
import requests

# Create directories
Path('models').mkdir(exist_ok=True)
Path('input_images').mkdir(exist_ok=True)
Path('output_images').mkdir(exist_ok=True)

# Download your trained YOLO model from GitHub
YOLO_MODEL_URL = "https://github.com/yekcel/AI-Agent-MRI-Image-Defacing/blob/e5443a0bd32dcb86bf21142311e8d8231e1bef61/model_outputs/best_model.pt"
MODEL_PATH = "models/yolov8n-face.pt"

if not os.path.exists(MODEL_PATH):
    print("Downloading YOLO model...")
    response = requests.get(YOLO_MODEL_URL)
    with open(MODEL_PATH, 'wb') as f:
        f.write(response.content)
    print("✅ Model downloaded!")
else:
    print("✅ Model already exists!")


✅ Model downloaded!


In [4]:
import google.generativeai as genai
from google.colab import userdata

# Get API key from Colab secrets
GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)

print("✅ Gemini API configured!")


✅ Gemini API configured!


In [5]:
import cv2
import numpy as np

class IrreversibleDefacer:

    @staticmethod
    def expand_bbox(bbox, image_shape, expansion=1.2):
        x1, y1, x2, y2 = bbox
        h, w = image_shape[:2]

        center_x = (x1 + x2) / 2
        center_y = (y1 + y2) / 2
        width = x2 - x1
        height = y2 - y1

        new_width = width * expansion
        new_height = height * expansion

        new_x1 = max(0, int(center_x - new_width / 2))
        new_y1 = max(0, int(center_y - new_height / 2))
        new_x2 = min(w, int(center_x + new_width / 2))
        new_y2 = min(h, int(center_y + new_height / 2))

        return (new_x1, new_y1, new_x2, new_y2)

    @staticmethod
    def inpaint_deface(image, bbox):
        """Gentle: Inpainting with surrounding texture"""
        img = image.copy()
        x1, y1, x2, y2 = IrreversibleDefacer.expand_bbox(bbox, img.shape, 1.2)

        mask = np.zeros(img.shape[:2], dtype=np.uint8)
        mask[y1:y2, x1:x2] = 255

        result = cv2.inpaint(img, mask, inpaintRadius=3, flags=cv2.INPAINT_TELEA)
        return result

    @staticmethod
    def random_texture_deface(image, bbox):
        """Moderate: Random texture from background"""
        img = image.copy()
        x1, y1, x2, y2 = IrreversibleDefacer.expand_bbox(bbox, img.shape, 1.3)

        h, w = img.shape[:2]
        face_h, face_w = y2 - y1, x2 - x1

        bg_regions = []
        if y1 > face_h:
            bg_regions.append(img[y1-face_h:y1, :])
        if x1 > face_w:
            bg_regions.append(img[:, x1-face_w:x1])

        if bg_regions:
            bg_pixels = np.concatenate([r.flatten() for r in bg_regions])
            random_texture = np.random.choice(bg_pixels, size=(face_h, face_w))
        else:
            random_texture = np.random.randint(0, 256, size=(face_h, face_w), dtype=np.uint8)

        img[y1:y2, x1:x2] = random_texture
        return img

    @staticmethod
    def blackbox_deface(image, bbox):
        """Maximum: Complete black box"""
        img = image.copy()
        x1, y1, x2, y2 = IrreversibleDefacer.expand_bbox(bbox, img.shape, 1.4)
        img[y1:y2, x1:x2] = 0
        return img

    @staticmethod
    def skull_strip_deface(image, bbox):
        """Nuclear: Remove entire head region"""
        img = image.copy()
        x1, y1, x2, y2 = IrreversibleDefacer.expand_bbox(bbox, img.shape, 2.0)
        img[y1:y2, x1:x2] = 0
        return img

print("✅ Defacing strategies defined!")


✅ Defacing strategies defined!


In [15]:
import requests
from pathlib import Path

print("📥 Downloading YOLO models...")

# IMPORTANT: Use RAW links from your GitHub repo
DETECTOR_URL = "https://github.com/yekcel/AI-Agent-MRI-Image-Defacing/raw/main/PATH_TO_YOUR_DETECTOR_MODEL.pt"
VALIDATOR_URL = "https://github.com/yekcel/AI-Agent-MRI-Image-Defacing/raw/main/PATH_TO_YOUR_VALIDATOR_MODEL.pt"

def download_model(url, filename):
    try:
        print(f"   Downloading {filename}...")
        response = requests.get(url, stream=True)
        response.raise_for_status()

        # Check if it's actually a PT file
        content_type = response.headers.get('content-type', '')
        if 'html' in content_type.lower():
            raise ValueError(f"❌ URL returned HTML instead of model file! Check your link.")

        with open(filename, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)

        # Verify file size
        size = Path(filename).stat().st_size
        if size < 1000:  # PT files should be at least a few KB
            raise ValueError(f"❌ Downloaded file is too small ({size} bytes)")

        print(f"   ✅ {filename} downloaded ({size:,} bytes)")

    except Exception as e:
        print(f"   ❌ Error downloading {filename}: {str(e)}")
        raise

download_model(DETECTOR_URL, "yolo_face_detector.pt")
download_model(VALIDATOR_URL, "yolo_face_validator.pt")

print("\n✅ All models downloaded successfully!")


📥 Downloading YOLO models...
   ❌ Error downloading yolo_face_detector.pt: 404 Client Error: Not Found for url: https://github.com/yekcel/AI-Agent-MRI-Image-Defacing/raw/main/PATH_TO_YOUR_DETECTOR_MODEL.pt


HTTPError: 404 Client Error: Not Found for url: https://github.com/yekcel/AI-Agent-MRI-Image-Defacing/raw/main/PATH_TO_YOUR_DETECTOR_MODEL.pt

In [16]:
import requests

print("🔍 Checking repository structure...\n")

# Check main branch structure
api_url = "https://api.github.com/repos/yekcel/AI-Agent-MRI-Image-Defacing/contents/"

try:
    response = requests.get(api_url)
    response.raise_for_status()

    items = response.json()

    print("📁 Root directory contents:")
    for item in items:
        print(f"   {'📁' if item['type'] == 'dir' else '📄'} {item['name']}")

    # Check if there's a models folder
    models_url = "https://api.github.com/repos/yekcel/AI-Agent-MRI-Image-Defacing/contents/models"
    response = requests.get(models_url)

    if response.status_code == 200:
        print("\n📁 Contents of /models/ folder:")
        models = response.json()
        for model in models:
            if model['name'].endswith('.pt'):
                print(f"   ✅ {model['name']} ({model['size']:,} bytes)")
                print(f"      Raw URL: {model['download_url']}")
    else:
        print("\n⚠️  No /models/ folder found in repository")

except Exception as e:
    print(f"❌ Error: {e}")


🔍 Checking repository structure...

📁 Root directory contents:
   📄 .gitignore
   📄 LICENSE
   📄 MRI-Face-Detection-Model.ipynb
   📄 Multi_agnet_deface_MRI.ipynb
   📄 README.md
   📁 model_outputs
   📁 test_data

⚠️  No /models/ folder found in repository


In [7]:
class DefacerAgent:
    def __init__(self):
        self.name = "DefacerAgent"
        self.strategies = {
            'gentle': IrreversibleDefacer.inpaint_deface,
            'moderate': IrreversibleDefacer.random_texture_deface,
            'maximum': IrreversibleDefacer.blackbox_deface,
            'skull_strip': IrreversibleDefacer.skull_strip_deface
        }

    def run(self, image, detections, strategy='gentle'):
        """Apply defacing strategy to detected faces"""
        defaced_image = image.copy()

        for detection in detections:
            bbox = detection['bbox']
            deface_func = self.strategies[strategy]
            defaced_image = deface_func(defaced_image, bbox)

        return {
            'defaced_image': defaced_image,
            'strategy_used': strategy,
            'status': 'success'
        }

print("✅ DefacerAgent ready!")


✅ DefacerAgent ready!


In [8]:
class ValidatorAgent:
    def __init__(self, model_path='models/yolov8n-face.pt', threshold=0.3):
        self.yolo = YOLO(model_path)
        self.threshold = threshold
        self.name = "ValidatorAgent"

    def run(self, image):
        """Validate defacing quality - good enough if confidence < threshold"""
        results = self.yolo(image, conf=0.1, verbose=False)

        if len(results[0].boxes) == 0:
            return {
                'is_valid': True,
                'max_confidence': 0.0,
                'num_faces': 0,
                'message': 'No face detected - Perfect!',
                'status': 'success'
            }

        max_confidence = float(results[0].boxes.conf.max())
        is_valid = max_confidence < self.threshold

        return {
            'is_valid': is_valid,
            'max_confidence': max_confidence,
            'num_faces': len(results[0].boxes),
            'message': f"Max confidence: {max_confidence:.3f} ({'Valid' if is_valid else 'Need retry'})",
            'status': 'success'
        }

print("✅ ValidatorAgent ready!")


✅ ValidatorAgent ready!


In [9]:
class GeminiStrategyAgent:
    def __init__(self):
        self.model = genai.GenerativeModel('gemini-2.0-flash-exp')
        self.name = "GeminiStrategyAgent"

    def run(self, validation_result, iteration):
        """Decide next defacing strategy using Gemini"""
        confidence = validation_result.get('max_confidence', 0)
        num_faces = validation_result.get('num_faces', 0)

        prompt = f"""You are a defacing strategy optimizer for medical MRI images.

Current Status:
- Iteration: {iteration}/4
- Remaining faces detected: {num_faces}
- Max detection confidence: {confidence:.3f}

Available strategies (from gentle to aggressive):
1. gentle: Inpainting (natural texture fill)
2. moderate: Random texture from background
3. maximum: Complete black box
4. skull_strip: Nuclear option - removes entire head region

Decision Rules:
- If confidence > 0.7 → Use "maximum"
- If confidence 0.4-0.7 → Use "moderate"
- If confidence < 0.4 → Use "gentle"
- If iteration >= 3 → Use "skull_strip"

Respond with ONLY ONE WORD - the strategy name (gentle/moderate/maximum/skull_strip).
"""

        try:
            response = self.model.generate_content(prompt)
            strategy = response.text.strip().lower()

            valid_strategies = ['gentle', 'moderate', 'maximum', 'skull_strip']
            if strategy not in valid_strategies:
                strategy = 'moderate'

            return {
                'strategy': strategy,
                'reasoning': f"Confidence {confidence:.3f} at iteration {iteration}",
                'status': 'success'
            }
        except Exception as e:
            return {
                'strategy': 'moderate',
                'reasoning': f"Error: {str(e)} - using fallback",
                'status': 'fallback'
            }

print("✅ GeminiStrategyAgent ready!")


✅ GeminiStrategyAgent ready!


In [10]:
class MultiAgentOrchestrator:
    def __init__(self, max_iterations=4, validation_threshold=0.3):
        self.detector = DetectorAgent()
        self.defacer = DefacerAgent()
        self.validator = ValidatorAgent(threshold=validation_threshold)
        self.strategy_agent = GeminiStrategyAgent()
        self.max_iterations = max_iterations

        print(f"🎯 Orchestrator initialized with {max_iterations} max iterations")

    def run(self, image_path):
        """Main orchestration loop"""
        print(f"\n{'='*60}")
        print(f"🚀 Processing: {Path(image_path).name}")
        print(f"{'='*60}\n")

        # Step 1: Load image
        image = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)
        if image is None:
            return {'status': 'error', 'message': 'Failed to load image'}

        # Step 2: Detect faces
        print(f"[{self.detector.name}] Detecting faces...")
        detection_result = self.detector.run(image)

        if detection_result['num_faces'] == 0:
            print("✅ No faces detected - image is clean!")
            return {
                'status': 'no_faces_detected',
                'message': 'No faces found in image',
                'defaced_image': image,
                'iterations': 0,
                'strategies_used': []
            }

        print(f"   Found {detection_result['num_faces']} face(s)")

        # Step 3: Iterative defacing loop
        detections = detection_result['detections']
        current_image = image.copy()
        strategy_history = []

        for iteration in range(1, self.max_iterations + 1):
            print(f"\n--- Iteration {iteration}/{self.max_iterations} ---")

            # Decide strategy
            if iteration == 1:
                strategy = 'gentle'
                print(f"[{self.strategy_agent.name}] Starting with '{strategy}' strategy")
            else:
                print(f"[{self.strategy_agent.name}] Analyzing validation result...")
                strategy_result = self.strategy_agent.run(validation_result, iteration)
                strategy = strategy_result['strategy']
                print(f"   Decision: '{strategy}' ({strategy_result['reasoning']})")

            strategy_history.append(strategy)

            # Apply defacing
            print(f"[{self.defacer.name}] Applying '{strategy}' defacing...")
            deface_result = self.defacer.run(current_image, detections, strategy)
            current_image = deface_result['defaced_image']

            # Validate result
            print(f"[{self.validator.name}] Validating defaced image...")
            validation_result = self.validator.run(current_image)
            print(f"   {validation_result['message']}")

            if validation_result['is_valid']:
                print(f"\n✅ SUCCESS! Defacing completed in {iteration} iteration(s)")
                break
        else:
            print(f"\n⚠️  Reached max iterations ({self.max_iterations})")

        return {
            'status': 'success' if validation_result['is_valid'] else 'max_iterations_reached',
            'iterations': iteration,
            'strategies_used': strategy_history,
            'final_confidence': validation_result['max_confidence'],
            'final_faces_detected': validation_result['num_faces'],
            'defaced_image': current_image
        }

print("✅ MultiAgentOrchestrator ready!")


✅ MultiAgentOrchestrator ready!


In [12]:

TEST_IMAGE_URLS = [
    "https://github.com/yekcel/AI-Agent-MRI-Image-Defacing/tree/main/test_data/images/dicom_000209_jpg.rf.68d7b19c0c5017332d18516147668c4f.jpg",
    "https://github.com/yekcel/AI-Agent-MRI-Image-Defacing/tree/main/test_data/images/dicom_000218_jpg.rf.2d70d50536d38f2bfa4bd4699955db8b.jpg",
    "https://github.com/yekcel/AI-Agent-MRI-Image-Defacing/tree/main/test_data/images/dicom_002284_jpg.rf.0ef17c3c1c6d85b222e00e3ec887c8ce.jpg",
]

print("📥 Downloading test images...")

for idx, url in enumerate(TEST_IMAGE_URLS, 1):
    filename = url.split('/')[-1]
    print(f"   [{idx}/{len(TEST_IMAGE_URLS)}] {filename}")

    response = requests.get(url)
    with open(f"input_images/{filename}", 'wb') as f:
        f.write(response.content)

print(f"\n✅ Downloaded {len(TEST_IMAGE_URLS)} image(s)!")


📥 Downloading test images...
   [1/3] dicom_000209_jpg.rf.68d7b19c0c5017332d18516147668c4f.jpg
   [2/3] dicom_000218_jpg.rf.2d70d50536d38f2bfa4bd4699955db8b.jpg
   [3/3] dicom_002284_jpg.rf.0ef17c3c1c6d85b222e00e3ec887c8ce.jpg

✅ Downloaded 3 image(s)!


In [14]:
import os

# Check if model file exists and its size
model_path = "yolo_model.pt"

if os.path.exists(model_path):
    file_size = os.path.getsize(model_path)
    print(f"✅ Model file exists: {model_path}")
    print(f"📊 File size: {file_size:,} bytes ({file_size / 1024 / 1024:.2f} MB)")

    # Check first few bytes
    with open(model_path, 'rb') as f:
        first_bytes = f.read(20)
        print(f"🔍 First bytes (hex): {first_bytes[:10].hex()}")
        print(f"🔍 First byteascii): {first_bytes[:10]}")

    # Check if it's actually a text file (HTML error page)
    with open(model_path, 'rb') as f:
        content = f.read(200)
        if b'<' in content or

SyntaxError: invalid syntax (ipython-input-1465640661.py, line 20)

In [13]:
from PIL import Image
import matplotlib.pyplot as plt

# Initialize orchestrator
orchestrator = MultiAgentOrchestrator(max_iterations=4, validation_threshold=0.3)

# Process single image
image_path = 'input_images/YOUR_IMAGE_NAME.png'  # Change this
result = orchestrator.run(image_path)

# Display results
print(f"\n{'='*60}")
print(f"📊 FINAL RESULTS")
print(f"{'='*60}")
print(f"Status: {result['status']}")
print(f"Iterations: {result['iterations']}")
print(f"Strategies used: {' → '.join(result['strategies_used'])}")
print(f"Final confidence: {result['final_confidence']:.3f}")
print(f"Final faces detected: {result.get('final_faces_detected', 0)}")

# Save and display
output_path = f"output_images/defaced_{Path(image_path).name}"
cv2.imwrite(output_path, result['defaced_image'])
print(f"\n💾 Saved to: {output_path}")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].imshow(cv2.imread(image_path, cv2.IMREAD_GRAYSCALE), cmap='gray')
axes[0].set_title('Original')
axes[0].axis('off')

axes[1].imshow(result['defaced_image'], cmap='gray')
axes[1].set_title(f"Defaced ({result['iterations']} iterations)")
axes[1].axis('off')

plt.tight_layout()
plt.show()


UnpicklingError: invalid load key, '\x0a'.